In [1]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import numpy as np
import pandas as pd
import pickle

import sklearn
from sklearn.model_selection import train_test_split

import os

# Implementation of a sentiment analysis

In [2]:
tf.enable_eager_execution()

In [3]:
dirname = os.getcwd()
dirname = os.path.dirname(dirname)
dataset_path = os.path.join(dirname, 'datasets/')
print(dataset_path)

/media/kandy/hdd/master-thesis/constituency-parsing/datasets/


In [3]:
gloveSet = pd.read_csv(dataset_path+'glove.42B.10d.txt', sep=' ', header=None)
print(gloveSet.shape)
print(gloveSet.head())

gloveWords = gloveSet.iloc[:,0:1]
gloveVectors = gloveSet.iloc[:,1:]

(1783088, 11)
    0        1         2         3         4         5         6       7   \
0    ,  0.18378 -0.121230 -0.119870  0.015227 -0.191210 -0.066074 -2.9876   
1  the -0.20838 -0.149320 -0.017528 -0.028432 -0.060104 -0.264600 -4.1445   
2    .  0.10876  0.002244  0.222130 -0.121020 -0.048959  0.018135 -3.8174   
3  and -0.09611 -0.257880 -0.358600 -0.328870  0.579500 -0.517740 -4.1582   
4   to -0.24837 -0.454610  0.039227 -0.284220 -0.031852  0.263550 -4.6323   

         8         9         10  
0  0.807950  0.067338 -0.131840  
1  0.629320  0.336720 -0.433950  
2 -0.032631 -0.625940 -0.518980  
3 -0.113710 -0.108480 -0.488850  
4  0.013890 -0.539280 -0.084454  


In [4]:
gloveCorpus = set()
gloveWords.iloc[:,0].str.lower().apply(gloveCorpus.add)
print(len(gloveCorpus))

glove_corpus_word_to_int = {}
glove_corpus_int_to_word = {}
for word in gloveCorpus:
    temp = len(glove_corpus_word_to_int)
    glove_corpus_word_to_int[word] = temp
    glove_corpus_int_to_word[temp] = word
#print(imdb_corpus_word_to_int)
#print(imdb_corpus_int_to_word)

UNK = '<unk>'
temp = len(glove_corpus_word_to_int)


outfile = glovePath +'glove_word_corpus.pic'
with open(outfile, 'wb') as pickle_file:
    pickle.dump([gloveCorpus,glove_corpus_word_to_int, glove_corpus_int_to_word], pickle_file)
    
#with open(outfile, 'rb') as pickle_file:    
#   loaded_data1 = pickle.load(pickle_file)

1783088


In [5]:
sentiment_dataset = pd.read_csv(dataset_path +'imdb_labelled.txt', sep='\t', header=None)
sentiment_dataset.rename({0:'comments',1:'sentiment'}, axis='columns', inplace=True) 
print(sentiment_dataset.shape)
print(sentiment_dataset.head())

(748, 2)
                                            comments  sentiment
0  A very, very, very slow-moving, aimless movie ...          0
1  Not sure who was more lost - the flat characte...          0
2  Attempting artiness with black & white and cle...          0
3       Very little music or anything to speak of.            0
4  The best scene in the movie was when Gerardo i...          1


In [6]:
def convert_imdb_corpus_into_int(sentence):
    words = sentence.lower().split()
    words = [word if word in gloveCorpus else UNK for word in words]
    words_to_num = [glove_corpus_word_to_int[word] for word in words]
    return words_to_num

sentiment_dataset_num = sentiment_dataset.copy()
sentiment_dataset_num['comments'] = sentiment_dataset['comments'].apply(convert_imdb_corpus_into_int)
sentiment_dataset_num.head()

,comments,sentiment
0,"[1144356, 349063, 349063, 349063, 349063, 3490...",0
1,"[349063, 349063, 349063, 349063, 349063, 34906...",0
2,"[349063, 550049, 349063, 349063, 349063, 34906...",0
3,"[349063, 349063, 349063, 349063, 349063, 29331...",0
4,"[1121171, 349063, 349063, 1040628, 1121171, 34...",1


In [7]:
imdb_train, imdb_test = sklearn.model_selection.train_test_split(sentiment_dataset_num, test_size=0.2)

In [8]:
print(imdb_train.head())
print(imdb_train.shape)

print(imdb_test.head())
print(imdb_test.shape)

                                              comments  sentiment
622  [349063, 883740, 349063, 349063, 1040628, 1144...          1
600  [349063, 349063, 349063, 684442, 349063, 29331...          1
525  [1372169, 349063, 349063, 349063, 349063, 7881...          0
66   [349063, 349063, 349063, 349063, 349063, 34906...          0
9    [349063, 1121171, 349063, 788117, 349063, 3490...          1
(598, 2)
                                              comments  sentiment
514  [349063, 684442, 349063, 349063, 349063, 34906...          1
93            [349063, 349063, 349063, 349063, 349063]          0
411  [1623186, 349063, 349063, 349063, 1040628, 349...          0
542  [349063, 349063, 349063, 349063, 349063, 68444...          1
304  [644091, 349063, 1144356, 349063, 349063, 6844...          1
(150, 2)


In [9]:
unkCount = 0
totalCount = 0
def count_UNK(token_list):
    global unkCount, totalCount
    match = glove_corpus_word_to_int[UNK]
    #print(token_list)
    #print(match)
    unkCount += token_list.count(match)
    totalCount += len(token_list)
    
sentiment_dataset_num['comments'].apply(count_UNK)
print(totalCount)
print(unkCount)

14602
11261


In [37]:
model = tf.keras.Sequential()
#model.add(tf.keras.layers.Embedding(input_dim=, output_dim=(gloveVectors.shape[1],), 
#                                    embeddings_initializer=embeddings))
model.add(tf.keras.layers.Dense(10, activation=tf.nn.tanh, input_shape=(10,)))
model.add(tf.keras.layers.Dense(10, activation=tf.nn.tanh))

model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.compile(optimizer=tf.train.AdamOptimizer(), loss=tf.losses.sigmoid_cross_entropy, metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_8 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 11        
Total params: 231
Trainable params: 231
Non-trainable params: 0
_________________________________________________________________


In [38]:
tensor_dataset = tf.random_uniform(shape=(1000, 10), maxval=10)
tensor_class = tf.random_uniform(shape=(1000, 1), maxval=2, dtype=tf.int32)
tensor_class =  tf.cast(tensor_class, dtype=tf.float32)
#print(tensor_dataset)

In [39]:
print(tensor_dataset.shape)
print(tensor_class.shape)

#print(cbow_words_avg.shape)
#print(classLabel.shape)

(1000, 10)
(1000, 1)


In [40]:
training_output = model.fit(x=tensor_dataset, y=tensor_class, epochs=1000)
print(training_output.history)

Epoch 1/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.7525 - acc: 0.4990
Epoch 2/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.7308 - acc: 0.4941
Epoch 3/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.7148 - acc: 0.5127
Epoch 4/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.7064 - acc: 0.5273
Epoch 5/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.7019 - acc: 0.5273
Epoch 6/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.6993 - acc: 0.5273
Epoch 7/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.6979 - acc: 0.5273
Epoch 8/1000
32/32 [==============================] - 0s 7ms/step - loss: 0.6968 - acc: 0.5273
Epoch 9/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.6962 - acc: 0.5273
Epoch 10/1000
32/32 [==============================] - 0s 7ms/step - loss: 0.6957 - acc: 0.5273
Epoch 11/1000
32/32 [============================

32/32 [==============================] - 0s 8ms/step - loss: 0.6629 - acc: 0.6162
Epoch 171/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.6629 - acc: 0.6162
Epoch 172/1000
32/32 [==============================] - 0s 7ms/step - loss: 0.6628 - acc: 0.6162
Epoch 173/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.6626 - acc: 0.6162
Epoch 174/1000
32/32 [==============================] - 0s 7ms/step - loss: 0.6626 - acc: 0.6172
Epoch 175/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.6624 - acc: 0.6172
Epoch 176/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.6624 - acc: 0.6182
Epoch 177/1000
32/32 [==============================] - 0s 7ms/step - loss: 0.6622 - acc: 0.6182
Epoch 178/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.6622 - acc: 0.6182
Epoch 179/1000
32/32 [==============================] - 0s 7ms/step - loss: 0.6622 - acc: 0.6182
Epoch 180/1000
32/32 [=======================

32/32 [==============================] - 0s 7ms/step - loss: 0.6568 - acc: 0.6260
Epoch 339/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.6570 - acc: 0.6260
Epoch 340/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.6568 - acc: 0.6260
Epoch 341/1000
32/32 [==============================] - 0s 7ms/step - loss: 0.6569 - acc: 0.6260
Epoch 342/1000
32/32 [==============================] - 0s 7ms/step - loss: 0.6571 - acc: 0.6260
Epoch 343/1000
32/32 [==============================] - 0s 7ms/step - loss: 0.6602 - acc: 0.6182
Epoch 344/1000
32/32 [==============================] - 0s 7ms/step - loss: 0.6600 - acc: 0.6191
Epoch 345/1000
32/32 [==============================] - 0s 7ms/step - loss: 0.6587 - acc: 0.6221
Epoch 346/1000
32/32 [==============================] - 0s 7ms/step - loss: 0.6581 - acc: 0.6240
Epoch 347/1000
32/32 [==============================] - 0s 7ms/step - loss: 0.6571 - acc: 0.6260
Epoch 348/1000
32/32 [=======================

32/32 [==============================] - 0s 7ms/step - loss: 0.6583 - acc: 0.6230
Epoch 507/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.6600 - acc: 0.6182
Epoch 508/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.6583 - acc: 0.6240
Epoch 509/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.6589 - acc: 0.6211
Epoch 510/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.6578 - acc: 0.6240
Epoch 511/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.6568 - acc: 0.6260
Epoch 512/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.6565 - acc: 0.6270
Epoch 513/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.6564 - acc: 0.6270
Epoch 514/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.6564 - acc: 0.6260
Epoch 515/1000
32/32 [==============================] - 0s 7ms/step - loss: 0.6564 - acc: 0.6260
Epoch 516/1000
32/32 [=======================

32/32 [==============================] - 0s 8ms/step - loss: 0.6555 - acc: 0.6289
Epoch 675/1000
32/32 [==============================] - 0s 9ms/step - loss: 0.6555 - acc: 0.6289
Epoch 676/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.6555 - acc: 0.6289
Epoch 677/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.6555 - acc: 0.6289
Epoch 678/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.6555 - acc: 0.6289A: 0s - loss: 0.6517 - acc: 0.6
Epoch 679/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.6555 - acc: 0.6289
Epoch 680/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.6555 - acc: 0.6289
Epoch 681/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.6555 - acc: 0.6289
Epoch 682/1000
32/32 [==============================] - 0s 9ms/step - loss: 0.6555 - acc: 0.6289
Epoch 683/1000
32/32 [==============================] - 0s 9ms/step - loss: 0.6555 - acc: 0.6289
Epoch 684/1000

32/32 [==============================] - 0s 9ms/step - loss: 0.6555 - acc: 0.6289
Epoch 843/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.6555 - acc: 0.6289
Epoch 844/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.6555 - acc: 0.6289
Epoch 845/1000
32/32 [==============================] - 0s 7ms/step - loss: 0.6555 - acc: 0.6289
Epoch 846/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.6555 - acc: 0.6289
Epoch 847/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.6555 - acc: 0.6289
Epoch 848/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.6555 - acc: 0.6289
Epoch 849/1000
32/32 [==============================] - 0s 7ms/step - loss: 0.6555 - acc: 0.6289
Epoch 850/1000
32/32 [==============================] - 0s 8ms/step - loss: 0.6555 - acc: 0.6289
Epoch 851/1000
32/32 [==============================] - 0s 9ms/step - loss: 0.6555 - acc: 0.6289
Epoch 852/1000
32/32 [=======================

In [48]:
class customModel(tf.keras.Model):
    def __init__(self, input_size):
        super(customModel,self).__init__()
        self.dense1 = tf.keras.layers.Dense(units=1, activation=tf.sigmoid, input_shape=(input_size,))
        
        
    def call(self, inputs):
        #print('Forward passing : ',inputs)
        output = self.dense1(inputs)
        #print('Output : ', output)
        return output

In [49]:
embeddings = tfe.Variable(name='embeddings', validate_shape= gloveVectors.shape, 
                          initial_value=gloveVectors.values, 
                          dtype=tf.float32, trainable=False)

custom = customModel(gloveVectors.shape[1])
#custom.summary()
custom.compile(optimizer=tf.train.AdamOptimizer(), loss=tf.losses.sigmoid_cross_entropy, metrics=['accuracy'])


In [50]:
imdb_train.shape

(598, 2)

In [51]:
epoch = 10
for i in range(epoch):
    print('Epoch : ', i+1)
    for j in range(imdb_train.shape[0]):
        #print(j)
        with tf.GradientTape() as tape:
            words = tf.nn.embedding_lookup(embeddings, imdb_train.iat[j,0])
            #print('Words shape : ', words.shape)
            cbow_words_avg = tf.math.reduce_mean(words, axis=0, keepdims=True)
            #cbow_words_avg =  tf.transpose(cbow_words_avg)
            #print('Average input : ', cbow_words_avg)
            #print(imdb_train.iat[j,0])
            classLabel = tf.convert_to_tensor(imdb_train.iat[j,1])
            classLabel = tf.reshape(classLabel, shape=(1,1))
            #print('class label : ', classLabel)
            custom_model_training_output = custom.fit(cbow_words_avg, classLabel)      
            #model_training_output = model.fit(cbow_words_avg, classLabel)

Epoch :  1
Epoch 1/1
1/1 [==============================] - 0s 49ms/step - loss: 0.4633 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 22ms/step - loss: 0.4543 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 7ms/step - loss: 1.0019 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 6ms/step - loss: 0.9797 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 8ms/step - loss: 0.4589 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 7ms/step - loss: 0.9941 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 8ms/step - loss: 0.4633 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 6ms/step - loss: 0.9886 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 9ms/step - loss: 0.9743 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 7ms/step - loss: 0.9897 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 9ms/step - loss: 0.4668 - acc: 1.00

Epoch 1/1
1/1 [==============================] - 0s 6ms/step - loss: 0.4761 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 8ms/step - loss: 0.9665 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 10ms/step - loss: 0.4784 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 10ms/step - loss: 0.9728 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 10ms/step - loss: 0.4726 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 9ms/step - loss: 0.9977 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 10ms/step - loss: 0.9738 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 10ms/step - loss: 0.4709 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 8ms/step - loss: 0.4691 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 8ms/step - loss: 0.9713 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 6ms/step - loss: 0.9686 - acc: 1.0000
Epoch

Epoch 1/1
1/1 [==============================] - 0s 10ms/step - loss: 0.4744 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 8ms/step - loss: 0.4759 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 11ms/step - loss: 0.4855 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 6ms/step - loss: 0.4777 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 10ms/step - loss: 0.9661 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 8ms/step - loss: 0.9616 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 9ms/step - loss: 0.4791 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 10ms/step - loss: 0.4730 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 9ms/step - loss: 0.4818 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 9ms/step - loss: 0.9603 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 7ms/step - loss: 0.9588 - acc: 1.0000
Epoch 

Epoch 1/1
1/1 [==============================] - 0s 7ms/step - loss: 0.9458 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 9ms/step - loss: 0.4885 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 11ms/step - loss: 0.9606 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 8ms/step - loss: 0.9582 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 9ms/step - loss: 0.4877 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 9ms/step - loss: 0.4918 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 6ms/step - loss: 0.9421 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 8ms/step - loss: 0.9695 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 8ms/step - loss: 0.9483 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 8ms/step - loss: 0.9469 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 9ms/step - loss: 0.4931 - acc: 1.0000
Epoch 1/1

Epoch 1/1
1/1 [==============================] - 0s 9ms/step - loss: 0.9467 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 6ms/step - loss: 0.4789 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 8ms/step - loss: 0.9574 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 7ms/step - loss: 0.9657 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 7ms/step - loss: 0.4830 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 6ms/step - loss: 0.9726 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 10ms/step - loss: 0.9356 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 9ms/step - loss: 0.9502 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 7ms/step - loss: 0.9368 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 8ms/step - loss: 0.9530 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 6ms/step - loss: 0.4773 - acc: 1.0000
Epoch 1/1

...

Epoch 1/1
1/1 [==============================] - 0s 10ms/step - loss: 0.5435 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 10ms/step - loss: 0.8645 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 7ms/step - loss: 0.8284 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 9ms/step - loss: 0.5456 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 10ms/step - loss: 0.5559 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 6ms/step - loss: 0.5432 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 8ms/step - loss: 0.8428 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 8ms/step - loss: 0.5714 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 8ms/step - loss: 0.8566 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 10ms/step - loss: 0.5743 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 8ms/step - loss: 0.8453 - acc: 1.0000
Epoch 

Epoch 1/1
1/1 [==============================] - 0s 6ms/step - loss: 0.5462 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 9ms/step - loss: 0.8417 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 8ms/step - loss: 0.8391 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 9ms/step - loss: 0.5767 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 7ms/step - loss: 0.8628 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 8ms/step - loss: 0.8573 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 8ms/step - loss: 0.8488 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 7ms/step - loss: 0.5582 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 12ms/step - loss: 0.8335 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 9ms/step - loss: 0.5772 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 9ms/step - loss: 0.5512 - acc: 1.0000
Epoch 1/1

Epoch 1/1
1/1 [==============================] - 0s 6ms/step - loss: 0.5606 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 6ms/step - loss: 0.5638 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 9ms/step - loss: 0.8582 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 7ms/step - loss: 0.5558 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 11ms/step - loss: 0.8679 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 8ms/step - loss: 0.5508 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 7ms/step - loss: 0.5582 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 6ms/step - loss: 0.8592 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 9ms/step - loss: 0.8546 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 9ms/step - loss: 0.5783 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 8ms/step - loss: 0.8332 - acc: 1.0000
Epoch 1/1

Epoch 1/1
1/1 [==============================] - 0s 10ms/step - loss: 0.5569 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 7ms/step - loss: 0.8285 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 8ms/step - loss: 0.5652 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 7ms/step - loss: 0.5809 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 10ms/step - loss: 0.5641 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 9ms/step - loss: 0.5628 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 9ms/step - loss: 0.8199 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 7ms/step - loss: 0.5564 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 7ms/step - loss: 0.5763 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 11ms/step - loss: 0.5532 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 10ms/step - loss: 0.5428 - acc: 1.0000
Epoch 

KeyboardInterrupt: 